In [2]:
# from PIL import Image, ImageSequence

In [3]:
# im = Image.open(r"C:\Users\parth\Downloads\37-059-26943-0000_PARTIAL WELL RECORD.tif")

# for i, page in enumerate(ImageSequence.Iterator(im)):
#     page.save("page%d.png" % i)

In [5]:
# im = Image.open(r"X:\Parth\Celebal\Abhishek Singh\Images\page0.png")

In [1]:
import time
import json
import pandas as pd
from requests import get, post
from IPython.display import display

In [8]:
endpoint = r"https://demoreconizer.cognitiveservices.azure.com/formrecognizer/v2.1/layout/analyze"
apim_key = r"19f1f4c6a6d84fb59a54f96f7b508325"

In [9]:
headers = {
    'Content-Type': 'image/png',
    'Ocp-Apim-Subscription-Key': apim_key,
}

params = {
    'readingOrder': 'natural'
}

In [10]:
path_to_image = r"X:\Parth\Celebal\Abhishek Singh\Images\page0.png"

with open(path_to_image, "rb") as f:
    data_bytes = f.read()

In [11]:
resp = post(url = endpoint, data = data_bytes, headers = headers, params=params)

if resp.status_code == 202:
    get_url = resp.headers["operation-location"]
    resp = get(url = resp.headers["operation-location"], headers = {"Ocp-Apim-Subscription-Key": apim_key})
    resp_json = json.loads(resp.text)

    while resp_json["status"] == "running":
        print('Waiting for result...')
        resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
        resp_json = json.loads(resp.text)
        time.sleep(5)

    if resp_json["status"] == "succeeded":
        with open(r"X:\Parth\Celebal\Abhishek Singh\temp.json", "w") as jfile:
            jfile.writelines(json.dumps(resp_json))
        

Waiting for result...
Waiting for result...


In [15]:
resp_json = json.load(open(r"X:\Parth\Celebal\Abhishek Singh\temp.json", "r"))

- lines

In [143]:
lines_dict = {}

for lines in resp_json["analyzeResult"]["readResults"][0]["lines"]:
    temp_bb = lines["boundingBox"]
    lines_dict[lines["text"]] = [(temp_bb[0], temp_bb[1]), (temp_bb[4], temp_bb[5])]

In [144]:
lines_dict

{'8000-FM-OOGM0004aU 10/2016': [(68, 20), (423, 43)],
 'Form': [(67, 46), (129, 67)],
 'pennsylvania': [(154, 78), (368, 109)],
 'DEPARTMENT OF ENVIRONMENTAL': [(156, 110), (410, 130)],
 'PROTECTION': [(156, 129), (254, 146)],
 'COMMONWEALTH OF PENNSYLVANIA': [(648, 100), (1067, 122)],
 'DEPARTMENT OF ENVIRONMENTAL PROTECTION': [(585, 123), (1129, 147)],
 'OFFICE OF OIL AND GAS MANAGEMENT': [(637, 150), (1077, 173)],
 'DEP USE ONLY': [(1314, 104), (1482, 125)],
 'Site ID': [(1247, 132), (1298, 148)],
 'Primary Facility Id': [(1202, 188), (1341, 207)],
 'Well Operator': [(73, 323), (195, 346)],
 'Rice Drilling B LLC': [(73, 355), (265, 378)],
 'Address': [(70, 384), (145, 405)],
 '2200 Rice Drive': [(71, 411), (234, 434)],
 'City': [(71, 445), (105, 465)],
 'Canonsburg': [(71, 467), (194, 491)],
 'Phone': [(72, 499), (129, 517)],
 'Well Record': [(740, 196), (969, 232)],
 '(Unconventional Operators Only)': [(590, 238), (1118, 278)],
 'NAD 83': [(1090, 407), (1160, 424)],
 'Email': [(451

- words

In [115]:
words_dict = {}

for lines in resp_json["analyzeResult"]["readResults"][0]["lines"]:
    for words in lines["words"]:
        temp_bb = words["boundingBox"]
        words_dict[words["text"]] = [(temp_bb[0], temp_bb[1]), (temp_bb[4], temp_bb[5])]

In [116]:
words_dict

{'8000-FM-OOGM0004aU': [(69, 21), (312, 44)],
 '10/2016': [(342, 21), (420, 44)],
 'Form': [(68, 47), (114, 68)],
 'pennsylvania': [(155, 80), (363, 110)],
 'DEPARTMENT': [(585, 124), (735, 147)],
 'OF': [(720, 150), (751, 173)],
 'ENVIRONMENTAL': [(777, 125), (974, 147)],
 'PROTECTION': [(979, 126), (1123, 148)],
 'COMMONWEALTH': [(649, 101), (839, 121)],
 'PENNSYLVANIA': [(887, 100), (1060, 123)],
 'OFFICE': [(637, 150), (714, 173)],
 'OIL': [(757, 150), (795, 173)],
 'AND': [(592, 1387), (648, 1413)],
 'GAS': [(854, 150), (901, 174)],
 'MANAGEMENT': [(909, 150), (1074, 174)],
 'DEP': [(640, 327), (676, 347)],
 'USE': [(1365, 104), (1409, 125)],
 'ONLY': [(1417, 104), (1477, 125)],
 'Site': [(1247, 133), (1274, 149)],
 'ID': [(1277, 133), (1293, 149)],
 'Primary': [(1203, 188), (1261, 207)],
 'Facility': [(1265, 188), (1319, 207)],
 'Id': [(1545, 190), (1561, 207)],
 'Well': [(688, 1482), (732, 1506)],
 'Operator': [(113, 325), (194, 346)],
 'Rice': [(124, 412), (171, 434)],
 'Drilli

In [145]:
lines_df = pd.DataFrame(lines_dict.items())

In [118]:
words_df = pd.DataFrame(words_dict.items())

In [122]:
words_df.head()

,0,1
0,8000-FM-OOGM0004aU,"[(69, 21), (312, 44)]"
1,10/2016,"[(342, 21), (420, 44)]"
2,Form,"[(68, 47), (114, 68)]"
3,pennsylvania,"[(155, 80), (363, 110)]"
4,DEPARTMENT,"[(585, 124), (735, 147)]"


In [123]:
words_df.shape

(313, 2)

In [146]:
lines_df.head()

,0,1
0,8000-FM-OOGM0004aU 10/2016,"[(68, 20), (423, 43)]"
1,Form,"[(67, 46), (129, 67)]"
2,pennsylvania,"[(154, 78), (368, 109)]"
3,DEPARTMENT OF ENVIRONMENTAL,"[(156, 110), (410, 130)]"
4,PROTECTION,"[(156, 129), (254, 146)]"


In [124]:
words_df[2] = words_df[1].apply(lambda x: ((x[0][0] + x[1][0]) / 2, ((x[0][1] + x[1][1]) / 2)))
words_df.head()

,0,1,2
0,8000-FM-OOGM0004aU,"[(69, 21), (312, 44)]","(190.5, 32.5)"
1,10/2016,"[(342, 21), (420, 44)]","(381.0, 32.5)"
2,Form,"[(68, 47), (114, 68)]","(91.0, 57.5)"
3,pennsylvania,"[(155, 80), (363, 110)]","(259.0, 95.0)"
4,DEPARTMENT,"[(585, 124), (735, 147)]","(660.0, 135.5)"


In [147]:
lines_df[2] = lines_df[1].apply(lambda x: ((x[0][0] + x[1][0]) / 2, ((x[0][1] + x[1][1]) / 2)))
lines_df.head()

,0,1,2
0,8000-FM-OOGM0004aU 10/2016,"[(68, 20), (423, 43)]","(245.5, 31.5)"
1,Form,"[(67, 46), (129, 67)]","(98.0, 56.5)"
2,pennsylvania,"[(154, 78), (368, 109)]","(261.0, 93.5)"
3,DEPARTMENT OF ENVIRONMENTAL,"[(156, 110), (410, 130)]","(283.0, 120.0)"
4,PROTECTION,"[(156, 129), (254, 146)]","(205.0, 137.5)"


In [148]:
lines_df.shape

(193, 3)

In [149]:
with open(r"X:\Parth\Celebal\Abhishek Singh\temp.txt", "r") as txt_file:
    stats = json.load(txt_file)

In [150]:
stats

[[0, 71, 1637, 2030, 533012],
 [0, 0, 1700, 2200, 870489],
 [106, 85, 17, 11, 129],
 [1158, 103, 86, 24, 2025],
 [1252, 103, 280, 25, 6695],
 [1540, 104, 84, 24, 1937],
 [105, 117, 3, 3, 9],
 [589, 127, 537, 17, 9129],
 [1157, 133, 232, 51, 11362],
 [1394, 135, 230, 50, 11249],
 [1157, 189, 231, 86, 19700],
 [1394, 191, 230, 85, 19479],
 [77, 292, 739, 29, 18216],
 [833, 295, 29, 26, 463],
 [824, 296, 1, 1, 1],
 [872, 296, 753, 29, 17810],
 [76, 325, 654, 116, 64011],
 [626, 328, 104, 53, 5423],
 [737, 329, 427, 55, 22642],
 [1169, 331, 257, 54, 13407],
 [1432, 332, 193, 113, 20403],
 [737, 388, 345, 25, 8266],
 [1088, 389, 76, 55, 4079],
 [1169, 389, 257, 55, 13751],
 [736, 417, 346, 26, 8264],
 [74, 444, 483, 51, 23351],
 [562, 447, 99, 48, 4651],
 [665, 447, 64, 49, 3036],
 [736, 447, 428, 50, 20523],
 [1169, 449, 455, 51, 22027],
 [74, 499, 655, 52, 31651],
 [736, 503, 426, 49, 20170],
 [1169, 503, 455, 52, 22111],
 [74, 555, 1550, 36, 43596],
 [554, 566, 13, 13, 169],
 [904, 567, 

In [65]:
for ele in stats:
    print(ele)
    break

[0, 21, 1636, 2079, 964307]


In [151]:
def mapping(x):
    flag = False
    for bb in stats[2:]:
        if (((x[2][0] >= bb[0]) and (x[2][0] <= (bb[0] + bb[2]))) and \
            (x[2][1] >= bb[1] and (x[2][1] <= (bb[1] + bb[3])))):
            flag = True
            break

    if flag == True:
        return True
    else:
        return False

In [152]:
final_df = pd.DataFrame([])

In [127]:
final_df["Table_Bool"] = words_df.apply(lambda x: mapping(x), axis=1)

In [130]:
words_df[final_df["Table_Bool"]]
#.to_csv(r"X:\Parth\Celebal\Abhishek Singh\temp2.csv")

,0,1,2
12,AND,"[(592, 1387), (648, 1413)]","(620.0, 1400.0)"
15,DEP,"[(640, 327), (676, 347)]","(658.0, 337.0)"
16,USE,"[(1365, 104), (1409, 125)]","(1387.0, 114.5)"
17,ONLY,"[(1417, 104), (1477, 125)]","(1447.0, 114.5)"
18,Site,"[(1247, 133), (1274, 149)]","(1260.5, 141.0)"
...,...,...,...
307,From,"[(746, 1863), (792, 1884)]","(769.0, 1873.5)"
308,To,"[(882, 1864), (904, 1883)]","(893.0, 1873.5)"
309,Pulled,"[(1366, 1865), (1429, 1886)]","(1397.5, 1875.5)"
310,Left,"[(1527, 1866), (1572, 1886)]","(1549.5, 1876.0)"


In [153]:
final_df["Table_Bool"] = lines_df.apply(lambda x: mapping(x), axis=1)

In [156]:
lines_df[final_df["Table_Bool"]]
#.to_csv(r"X:\Parth\Celebal\Abhishek Singh\temp1.csv")

,0,1,2
6,DEPARTMENT OF ENVIRONMENTAL PROTECTION,"[(585, 123), (1129, 147)]","(857.0, 135.0)"
8,DEP USE ONLY,"[(1314, 104), (1482, 125)]","(1398.0, 114.5)"
9,Site ID,"[(1247, 132), (1298, 148)]","(1272.5, 140.0)"
10,Primary Facility Id,"[(1202, 188), (1341, 207)]","(1271.5, 197.5)"
11,Well Operator,"[(73, 323), (195, 346)]","(134.0, 334.5)"
...,...,...,...
187,From,"[(745, 1863), (804, 1883)]","(774.5, 1873.0)"
188,To,"[(882, 1864), (911, 1883)]","(896.5, 1873.5)"
189,Pulled,"[(1365, 1865), (1434, 1885)]","(1399.5, 1875.0)"
190,Left,"[(1527, 1866), (1572, 1886)]","(1549.5, 1876.0)"


In [44]:
bb_df = pd.read_csv(r"X:\Parth\Celebal\Abhishek Singh\temp.csv")

In [45]:
bb_df.head()

,Unnamed: 0,0,1,2,3,4
0,0,0,21,1636,2079,964307
1,1,0,0,1700,2200,834119
2,2,174,29,4,5,19
3,3,192,29,4,5,20
4,4,209,31,1,1,1
